In [209]:
import time
import os

import numpy as np
import cv2
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import psutil
import pyautogui
from pywinauto import Application

### 필요한 변수선언

In [210]:
certificate_password = os.getenv("CERTIFICATE_PASSWORD")
driver_path = os.getenv("DRIVER_PATH")
dert_path = os.getenv('DERT_PATH')
key_path = os.getenv('KEY_PATH')

### 홈택스 페이지접근


In [211]:
options = Options()
# chrome user agent를 지정해주기
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36")
# chrome의 user-agent를 지정해준다.


driver = webdriver.Chrome(driver_path, options=options)
driver.implicitly_wait(10)
driver.get('https://www.hometax.go.kr/websquare/websquare.wq?w2xPath=/ui/pp/index.xml&tmIdx=0&tm2lIdx=&tm3lIdx=')

cookies = driver.get_cookies()

button_for_login = driver.find_element_by_xpath("//a[@id='group1544'][@class='w2group ']")
time.sleep(5)
driver.save_screenshot('hometax/current_page.png')

True

### 홈택스 로그인페이지 

In [212]:
button_for_login.click()
time.sleep(5)
driver.maximize_window()
driver.save_screenshot('hometax/current_page.png')

True

### 홈택스 공인인증서 창 클릭 

In [213]:
driver.switch_to.frame('txppIframe')
button_for_login = driver.find_element_by_xpath("//input[@type='button'][@id='trigger38'][@value='공인인증서 로그인']")
time.sleep(5)
button_for_login.click()
driver.save_screenshot('hometax/current_page.png')

True

### 인증서 로그인을 위한 클릭

In [214]:
time.sleep(5)
driver.switch_to.default_content()
driver.switch_to.frame('txppIframe')
driver.switch_to.frame("dscert")
certificate_type = driver.find_element_by_xpath("//a[@id='stg_web']") # 브라우저 공인인증서 방법으로 로그인합니다.
certificate_type.click() # 선택을 위한 클릭
search_certificate_file = driver.find_element_by_xpath("//button[@id='in_browser']") # 브라우저 공인인증서를 찾기위한 버튼
search_certificate_file.click()
driver.save_screenshot('hometax/current_page.png')

True

### 인증서 찾기

In [215]:
time.sleep(3)
input_for_cert_path = driver.find_element_by_xpath("//input[@id='filefile2']")
input_for_cert_path.click()
driver.save_screenshot('hometax/current_page.png')

True

### FILE PATH 세팅 

In [216]:
file_path = '\"' + dert_path + '\"  \"' + key_path + '\"' # cert 파일과 der파일을 한꺼번에 입력해주어야 하기 때문에, 이렇게 경로를 설정해준다. 

### Pywinauto를 위한 process id 얻기

### 구조 : chromedriver.exe(부모프로세스) -> chrome.exe 실행(자식프로세스) 

In [217]:
chrome_driver_process_id = driver.service.process.pid # 현재 구동되고있는 크롬드라이버의 process id를 얻는다.
chrome_driver_process = psutil.Process(chrome_driver_process_id)  # 크롬드라이버의 pid로 psutil의 process 객체를 생성한다.
print(chrome_driver_process)
children_pid = 0
for children in chrome_driver_process.children(recursive=False): # 크롬드라이버의 자식프로세스인 chrome.exe 프로세스의 id를 얻는다.
    children_pid = children.pid
    print(children)

psutil.Process(pid=1804, name='chromedriver.exe', started='14:39:53')
psutil.Process(pid=1064, name='chrome.exe', started='14:39:54')


In [218]:
chrome_application = Application(backend='uia').connect(process=children_pid) # chrome.exe의 pid로 Application 객체를 생성한다.
file_dialog = chrome_application.top_window() # Application객체에서 window specification 객체를 생성한다.
dialog = file_dialog.OpenDialog # 크롬에서 file dialog부분만을 얻어온다.
ok_button = file_dialog.child_window(title="Open", control_type="Button", auto_id="1") # 경로입력후 열기버튼객체

In [219]:
edit_file_path = dialog.child_window(title_re="File name.*", control_type="ComboBox").child_window(title_re="File name.*")
edit_file_path.set_focus() # set focus 메소드는 화면으로 띄워줌 ! (보장됨) AOT
pyautogui.typewrite(file_path) 
ok_button.click()

<uia_controls.ButtonWrapper - 'Open', Button, 3713041255950518931>

In [220]:
input_for_password = driver.find_element_by_xpath("//input[@type='password'][@id='add_browser_password']")
input_for_password.send_keys(certificate_password)
button_for_login = driver.find_element_by_xpath("//button[@type='button'][@id='btn_common_confirm']")
button_for_login.click()

### 조회/발급탭 클릭

In [221]:
time.sleep(1)

driver.switch_to.default_content()
inquiry_button = driver.find_element_by_xpath("//a[descendant::span[text()='조회/발급']][@class='w2group ']")
inquiry_button.click()

### 세금계산서 발행을 위한 정보가져오기 from excel

In [222]:
from openpyxl import load_workbook

wb = load_workbook(filename='hometax/hometax_세금계산서발행.xlsx', data_only=True) # hometax 폴더에 있는 hometax_세금계산서발행.xlsx을 엽니다. 
ws = wb['발급'] # excel sheet중 발급시트를 가져옵니다.


In [223]:
necessary_input_row = ws["A3:L3"]
tax_filing_info = ws["A7:H22"]

In [224]:
register_number = "" # 사업자등록번호
company_name = "" # 상호명
ceo_name = "" # 대표이름
company_address = "" # 회사주소
business_topic = "" # 업종
business_detail = "" # 설명
company_email_id = "" #  email_id
company_email_domain = "" # email_id_domain
issue_date = "" # O

for row in necessary_input_row:
    register_number = str(row[1].value)
    company_name = str(row[2].value)
    ceo_name = str(row[3].value)
    company_address = str(row[4].value)
    business_topic = str(row[5].value)
    business_detail = str(row[6].value)
    company_email_id = str(row[7].value).split('@')[0]
    company_email_domain = str(row[7].value).split('@')[1] 
    issue_date = str(row[8].value)

In [225]:
print(company_email_domain)

iteyes.co.kr


###  전자세금계산서 발행

In [226]:
time.sleep(2)
driver.switch_to.default_content()
driver.switch_to.frame('txppIframe')
issue_tab = driver.find_element_by_xpath("//a[@id][text()='발급']")
issue_tab.click()
issue_button = driver.find_element_by_xpath("//a[@id][text()='건별발급']")
issue_button.click()

In [227]:
driver.switch_to.default_content()
driver.switch_to.frame('txppIframe')
input_registration_number = driver.find_element_by_xpath("//input[@id='edtDmnrBsnoTop']") # 사업자등록번호 input
button_registration_number = driver.find_element_by_xpath("//input[@type='button'][@id='btnDmnrBsnoCnfrTop']") # 사업자 등록번호 input 후 확인버튼
input_counterpart_company_name = driver.find_element_by_xpath("//input[@id='edtDmnrTnmNmTop']") # 거래상대방 상호
input_counterpart_ceo_name = driver.find_element_by_xpath("//input[@id='edtDmnrRprsFnmTop']") # 거래상대방 대표이름
input_counterpart_address = driver.find_element_by_xpath("//input[@id='edtDmnrPfbAdrTop']")
input_counterpart_business_topic = driver.find_element_by_xpath("//input[@id='edtDmnrBcNmTop']")
input_counterpart_business_detail = driver.find_element_by_xpath("//input[@id='edtDmnrItmNmTop']")
input_counterpart_email_id = driver.find_element_by_xpath("//input[@id='edtDmnrMchrgEmlIdTop']") # 이메일 아이디
input_counterpart_email_domain = driver.find_element_by_xpath("//input[@id='edtDmnrMchrgEmlDmanTop']") # 이메일 도메인(naver.com)
input_issue_date = driver.find_element_by_xpath("//input[@id='calWrtDtTop_input']") # 작성일자
button_issue = driver.find_element_by_xpath("//input[@id='btnIsn']") # 발행하기버튼
# date : genEtxivLsatTop_0_edtLsatSplDdTop

In [228]:
input_registration_number.send_keys(register_number)
button_registration_number.click()
time.sleep(1)
driver.switch_to.alert.accept()

In [229]:
def clear_and_send_keys(input_tag, key):
    print(key)
    if type(key) is not str:
        key = ""
    elif key == "None":
        key = ""
        
    input_tag.clear()
    input_tag.send_keys(key)

In [2]:
clear_and_send_keys(input_counterpart_company_name, company_name)
clear_and_send_keys(input_counterpart_ceo_name, ceo_name)
clear_and_send_keys(input_counterpart_email_id, company_email_id)
clear_and_send_keys(input_counterpart_email_domain, company_email_domain)
clear_and_send_keys(input_counterpart_business_topic, business_topic)
clear_and_send_keys(input_counterpart_business_detail, business_detail)

NameError: name 'clear_and_send_keys' is not defined

### 가격, 수량정보 (For-loop 필요)

In [231]:
"""
input 창의 id가 genEtxivLsatTop_{index}__edtLsatSplDdTop 형식으로 되어있음.
genEtxivLsatTop_0__edtLsatSplDdTop
genEtxivLsatTop_1__edtLsatSplDdTop
genEtxivLsatTop_2__edtLsatSplDdTop
genEtxivLsatTop_3__edtLsatSplDdTop
...

""" 
def get_input_element_dynamically(index):
    input_specific_day = driver.find_element_by_xpath(f"//input[@id='genEtxivLsatTop_{index}_edtLsatSplDdTop']") #  발행일자 
    input_commodity_name = driver.find_element_by_xpath(f"//input[@id='genEtxivLsatTop_{index}_edtLsatNmTop']") # 상품이름
    input_criteria = driver.find_element_by_xpath(f"//input[@id='genEtxivLsatTop_{index}_edtLsatRszeNmTop']") # 상품 측정 척도 (개, kg 등)
    input_quantity = driver.find_element_by_xpath(f"//input[@id='genEtxivLsatTop_{index}_edtLsatQtyTop']") # 상품 개수
    input_price = driver.find_element_by_xpath(f"//input[@id='genEtxivLsatTop_{index}_edtLsatUtprcTop']") # 상품 가격
    input_value_of_supply = driver.find_element_by_xpath(f"//input[@id='genEtxivLsatTop_{index}_edtLsatSplCftTop']")
    input_tax_amount = driver.find_element_by_xpath(f"//input[@id='genEtxivLsatTop_{index}_edtLsatTxamtTop']")
    input_sales_detail = driver.find_element_by_xpath(f"//input[@id='genEtxivLsatTop_{index}_edtLsatRmrkCntnTop']")
    return (input_specific_day, 
            input_commodity_name, 
            input_criteria, 
            input_quantity, 
            input_price, 
            input_value_of_supply, 
            input_tax_amount,
            input_sales_detail
            )

In [1]:
# 일자	품목	규격	수량	단가	공급가액	세액	비고
# [0]   [1]   [2]   [3]   [4]   [5]         [6]   [7]
"""
row[0] : 일자정보
row[1] : 품목정보
row[2] : 규격
row[3] : 수량
row[4] : 단가
row[5] : 공급가액
row[6] : 세액
row[7] : 비고
"""

for index, row in enumerate(tax_filing_info):
    if row[1].value is None: # 품목이 없음면 loop를 종료한다.
        break 
        
    input_specific_day, input_commodity_name, input_criteria, input_quantity, input_price, input_value_of_supply, input_tax_amount, input_sales_detail = get_input_element_dynamically(index)
    clear_and_send_keys(input_specific_day, str(row[0].internal_value))
    clear_and_send_keys(input_commodity_name, str(row[1].value))
    clear_and_send_keys(input_criteria, str(row[2].value))
    clear_and_send_keys(input_quantity, str(row[3].value))
    clear_and_send_keys(input_price, str(row[4].value))
#     clear_and_send_keys(input_value_of_supply, str(row[5].internal_value))
#     clear_and_send_keys(input_tax_amount, str(row[6].internal_value))
    clear_and_send_keys(input_sales_detail, str(row[7].value))

NameError: name 'tax_filing_info' is not defined

In [233]:
button_for_issue = driver.find_element_by_xpath("//input[@id='btnIsn']")
button_for_issue.click()